# Introduction

Atala PRISM is an open-source decentralized identity solution built on the Cardano blockchain. PRISM builds upon the concepts of Distributed, Self-Sovereign Identity (DID, SSI) and Verifiable Credentials (VC) to provide a digital credentials product that empowers consumers to own and control their identity and personal information.

The Atala PRISM platform enables sharing of these credentials online, offline and on-chain, enabling a wide range of use cases across multiple business sectors. 

## Architecture
### Detailed component description
Atala PRISM consists of four major architectural layers of functionality: Decentralized Applications (dApps), Software Development Kit (SDK), Services (Internal and External), and Distributed Ledger Technology (DLT). Each layer is subdivided into different components:

> diagram

## dApps Layer
#### Mobile wallet
The Mobile Wallet is a mobile application available for Android and iOS whose purpose is to securely store verifiable credentials and other digital assets, and enable its owner (Holder) to receive and share them.

#### Management console
The Management Console is a web application designed for Issuers and Verifiers. The console enables users to manage the issuance and verification of credentials.

#### Interactive web demo
The Interactive Web Demo (reached from atalaprism.io) enables its users to experience what a smart city powered by Atala PRISM would look like. 
The demo simulates the credential issuance process and showcases situations where issuing a particular credential requires proof of existence of another credential.

#### Third-party dApps

## SDK Layer
* Decentralized identity
* Verifiable credentials
* Cryptography

## Services Layer
> Services layer to be described

## Internal Services
#### Layer2
This service implements a Layer-2 protocol used by PRISM to batch transactions and provide transaction scalability and throughput. 

The Layer-2 service was originally based on the Sidetree protocol by the Distributed Identity Foundation (DIF), but we have extended it to support not only Decentralized Identifiers (DIDs) but also Verifiable Credentials (VCs). 

Atala PRISM implements the third design iteration for the Layer-2 protocol. We will keep expanding its versatility to accommodate future deployment and operational use-cases.

#### Payments
The Atala PRISM Payments facility will enable the processing of payments in collaboration with a third-party payment processing provider. 

The Payments service includes core components whose role is to provide payments by both traditional- (credit- and debit- card based) and blockchain-based (cryptocurrencies, for example) means. This service will also model respective wallets and business rules. 

> Note: Payments service is currently in design phase.

#### Connector
The Connector service is responsible for secure, peer-to-peer (P2P) communication between PRISM users. This service is based on public-key cryptography. Peers (for instance, an Issuer of a Verifiable Credential and a future Holder of a Verifiable Credential) can only begin to exchange information after a channel has been established through the connector.

> **Note**: Note: Currently, the roles of Connector and Data Vault are merged, so that verifiable credentials are retained and can be recovered directly from the Connector.

#### Encrypted Data Vault (EDV)
The Atala PRISM Encrypted Data Vault (EDV) securely stores, retrieves, and restores credentials and other information. These operations are controlled by the owners of the information. 

The EDV is provided as a gRPC service with the following operations:
* Store: Stores a given encrypted payload, associating it with the authenticated user. A unique ID is returned.
* Get: Retrieves all encrypted payloads stored by the authenticated user, paginated and sorted by insertion order.

The EDV allows the Connector service to drop long-term storage while still allowing the mobile wallets to recover their data.

In terms of privacy, the EDV uses the same authorization infrastructure as the Connector, so communication is secure and the user is authenticated via its DID. On the client side, the payloads are encrypted with the user's DID, so bad actors cannot steal the information from either the wire or the server end. Because the server never actually sees raw data, there are no privacy concerns. The client can only query its own data, as the server filters payloads implicitly by the authenticated user. When a user queries for their payloads, only encrypted data will be returned and the client will need to decrypt them locally to use them.

## External Services
#### Oracles
The Oracles service will interface with the rest of PRISM either through its SDK or directly through smart contracts. 

> Note: Oracles service is currently in design phase.

#### KYC
A KYC service is responsible for implementing a KYC process.

> Note: KYC service is currently in design phase..

#### Distributed Ledger Technology (DLT)
This is both our trust anchor and a major provider of extended functionality, where applicable. For Bitcoin, we leverage its ability to store transaction metadata (through OP_RETURN), thus using it as a timestamping (or proof-of-ownership) service. Cardano, a more advanced blockchain, now provides us with metadata and will soon enable Smart Contract and Native Tokens capabilities. 

PRISM interfaces with blockchains in a plug-and-play fashion through a standardized READ/WRITE interface. This effectively means that a Bitcoin-based deployment and a Cardano-based deployment are architecturally and operationally quite similar in the majority of their components. The PRISM design can accommodate other blockchains too, but Cardano and Bitcoin are our current implementations for the DLT layer.

## Prism Node
The Atala Node is an application that follows the protocol's directives. The Node sends transactions with metadata to the Cardano blockchain, reads transactions confirmed by Cardano nodes, and interprets the relevant information from metadata in compliance with protocol rules. The Node includes an internal database that indexes the content from the Atala operations, which enables efficient data queries.

The Node requires the specification of a Genesis Cardano Block, so any prior operations will not affect the Node's state.

The Node state holds details such as:
* DIDs, and their respective current document.
* Credentials (hashes only), who issued them, and when they were revoked, if applicable

The Node state is updated by the Synchronizer. This component continuously looks for new blocks on the Cardano node, stores the Cardano block headers, and applies rollbacks when necessary.

> Diagram Atala Node

## Management console Container Service
This is a storage and record-keeping system for the organizations using Atala PRISM. All the personal information transmitted in the apps will be stored in the Console. This information might include contact details, credential data, etc.

The Management Console consists of two elements:
* A gRPC Remote Procedure Calls (gRPC) API on the backend
* A web application on the frontend

The Management Console is a service run externally by each institution, organization, government, or other agency using Atala PRISM, which will enable these entities to retain control of their private data. IOHK does not handle or store any of this information.

## Management Console database (Credentials store)
An SQL database that stores any data required by the Management Console backend. A dedicated database enables this data to be persistent.

#### Mobile Wallet
This component enables users to create and self-issue identities, receive and share credentials, and store private data (credentials information, DIDs, etc.) locally.

The wallet interacts with other SSI components and generates keys and identities locally, which are used to interact with private organizations, educational institutions, government bodies, etc. The user can send or receive credentials from any of these entities, but users retain control of their data because the information is stored in their phones. The user then decides if, when, and whom to share that information with.

The wallet interacts with the following components:
* Connector API
* Node API
* Vault API

## Wallet
Institutions, organizations, government, and agencies using Atala PRISM use the Browser Wallet, a user interface and web SDK that acts as an identity management store. This component handles the DIDs (and associated keys) to digitally sign any credentials, after the relevant entity has approved it.

The Browser Wallet stores the identities self-issued by institutions using PRISM. These identities are then published to the blockchain. The locally-generated identities are used to interact with the backend services when needed, providing the necessary information to authenticate users to our backend services.

The Browser also works as a sign-in mechanism for other websites, a functionality similar to the log-in facility with Google or Facebook buttons.

This component also exposes the necessary APIs to allow the MC sign credentials. Specifically, the Browser Wallet interacts with:
* Connector API
* Node API
* Management Console backend (Management console API)

A gRPC API that interacts with the Management Console, enabling the Console's frontend to store the data needed from the users (contacts, credentials to be issued,The API interacts with:
* MC Database
* Connector API
* Node API

> **Note**: The browser wallet is a browser extension that will initially be available on the Chrome web store. There are future plans to create an extension for Firefox.

## Management Console frontend
A web-based user interface for the Management Console that interacts with:
* Browser Wallet
* MC backend

## Connector service (Container)
#### Connector (P2P API)
A centralized service run by IOHK that allows different wallets to communicate with each other by sending data messages (mostly credentials) with end-to-end encryption.

This component is a gPRC API, similar to the DIDComm from the DIF, that enables secure peer-to-peer communication between contacts, who have the ability to accept or reject the request. If the connection request is accepted, contacts can share data and credentials securely.

The Connector interacts with the Connector Database.

#### Connector database
An SQL database that stores/queries data sent by the Connector. It keeps track of which contacts can communicate with other parties.

## Vault Service (Container)
#### Vault API
A centralized service that encrypts user data and stores it in the user's device.

Vault is a gRPC API that exposes other APIs to store or retrieve the data encrypted in the user's device, and allows the user to reconstruct this private data in other devices.

The Vault is a key component, as it enables users to recover their data on different devices, as long as the user knows the recovery phrase. If the user has lost, forgotten it, or is unaware of this phrase, there is no way for IOHK to recover the data.

The Vault API interacts with the Vault database.

#### Vault database
An SQL database required to act as the storage layer for the Vault API. 

## Node service (Container)
#### PRISM node (Slayer node API)
The Node service provides the 2nd layer protocol for posting DIDs and credentials to the Cardano blockchain. The data is queried and sent through the Slayer node API, which also queries the data.

The Slayer node API is the component that posts proofs to the blockchain, and also checks that certain data is present on the chain. The API constantly listens to new transactions in Cardano and checks if these transactions carry any PRISM metadata. If they do, the data is parsed to extract the related proofs, and stored on the SQL database to be queried later.

The Slayer API interacts with:
* Cardano-node
* db-sync
* Prism Node database

## PRISM node database (Slayer indexing API)
An SQL database that provides a storage mechanism to avoid computing the information on every request, and has necessary data to handle Cardano transactions that have not yet been confirmed.

## Cardano Wallet backend
This component enables us to post transactions to the Cardano blockchain. The wallet backend is a REST API that stores the required ada balance to post transactions. The Wallet backend interacts with the Cardano node.

## Cardano-db-sync
The cardano node stores only the blockchain itself and associated information needed to validate the blockchain. The node is a complex piece of software, so this minimalist approach helps reduce its complexity. In particular, the node does not provide a convenient query interface for historical information on the blockchain. This data service is provided by a separate component using an Structured Query Language (SQL) database.

The cardano-db-sync component uses a local node (via its node-to-client IPC protocol) to follow the blockchain and continuously reflect the chain data into a PostgreSQL database. This database then provides all of the functionality to efficiently and conveniently query the chain data.
The database is the basis for other applications that need a flexible way to query the chain data. 

There are also GraphQL and REST APIs that provide views onto this database. This gives developers writing applications that integrate with Cardano the option of two flexible query languages: GraphQL or SQL, depending on the developer’s technology stack preferences. The overall deployment for an application using the SQL interface consists of a Cardano node, the cardano-db-sync process, PostgreSQL, and the user application.

The PostgreSQL database is designed to be used in a read-only fashion by applications, with only the cardano-db-sync component writing to the database to keep it in sync with the chain. The database schema is highly normalized. Several useful database queries are provided as Postgres views.

## Cardano-node
A Unix Socket API that stores the ledger state.


## Prism SDK
The Atala PRISM SDK empowers software teams on major application platforms (Web/Javascript, JVM, Android, iOS) to not only use core Atala PRISM functionality for their apps but also to extend it with new business rules and user experiences. 

Different parts of the SDK provide functionality related to: 
* Identity 
* Credentials 
* Cryptographic functions 

Whenever we can, we generate code for a specific platform from a single code source. In particular, we leverage Scala and Scala.js to compile libraries coded in Scala for three out of the four supported platforms: Web/Javascript, JVM, and Android. 

> **Note**: We are currently designing major parts of the SDK and re-working Atala PRISM components to use it, in essence promoting some internal dogfooding. Also, once the Payments service design is complete, we expect to provide related functionality as part of the SDK.

## SDK: prism-crypto
This is the lowest level SDK component, which provides all the primitives required to deal with the underlying cryptographic operations, like:

* Key generation
* Encryption
* Digital signatures
* Hashing

## SDK: prism-identity
The prism-identity component is built on top of prism-crypto, exposing all the necessary stuff to deal with identities, it implements our deterministic DID generation protocol so that you are able to recover your DIDs from a 12-words mnemonic seed, some of the possibilities being:

* DID generation
* DID validation
* DID recovery

## SDK: prism-credentials
The prism-credentials component is built on top of prism-identity, providing the necessary functionality to deal with Verifiable credentials, like:

* Issuing
* Revocation
* Verification

## SDK: prism-node
The prism-node component is the client-side side for the Prism Node, providing the necessary APIs to interact with the Cardano Blockchain, like:

* Pushing data
* Querying the state for DIDs/VCs

## SDK: prism-connector
The prism-connector component is the client-side side for the Prism Connector server, providing the necessary APIs to interact with other participants, mainly used to:

* Request Verifiable Credentials
* Receive Verifiable Credentials
* Send Verifiable Credentials

## SDK: prism-wallet
The prism-wallet exposes the necessary JavaScript functions to allow the interaction from websites, everything is handled by by the actual Prism Wallet installed in the browser, some possibilities being:

* Authenticate into a web app by using the Prism Wallet DIDs
* Signing data with the Prism Wallet DIDs
* Encrypt data with the Prism Wallet DIDs
* Decrypt data with the Prism Wallet DIDs